# Gaussian Process - A Brief Overview

# References and other useful resource


### Theory  

* Rasmussen, C. E. and Williams, C. K. I. (2006). [Gaussian Processes for Machine Learning](http://www.gaussianprocess.org/gpml/).
MIT Press. 238, 245  
  

### Intuition  

* [Gaussian Process Models in Python](http://nbviewer.jupyter.org/github/fonnesbeck/gp_showdown/blob/master/GP%20Showdown.ipynb) by Chris Fonnesbeck  
* [Video Lecture](http://videolectures.net/mlss07_rasmussen_bigp/) by Carl Rasmussen  
* Link between Deep neural network and Gaussian Process, [Blog post](http://mlg.eng.cam.ac.uk/yarin/blog_3d801aa532c1ce.html) by  Yarin Gal

  
### Practical  

* [GPML](http://www.gaussianprocess.org/gpml/code/matlab/doc/)  
* Using [scikit-learn](http://scikit-learn.org/stable/modules/gaussian_process.html)  
* GP in Stan see [Manual Chapter 17](https://github.com/stan-dev/stan/releases/download/v2.14.0/stan-reference-2.14.0.pdf) and [Stan example](https://github.com/stan-dev/example-models/tree/master/misc/gaussian-process). Also some information on [Prior choice](https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations#priors-for-gaussian-processes)   
* [GPy](https://github.com/sheffieldml/gpy) from Sheffield machine learning group  
* [GPflow](https://github.com/GPflow/GPflow)  
* [PyMC3 GP module](http://pymc-devs.github.io/pymc3/examples.html#gaussian-processes)  